In [ ]:
# TESTING JUPYTER NOTEBOOK INTEGRATION WITH KINESINLMS
# Using portion of python notebook found online by Steve Tuttle https://github.com/SteveTuttle/hawaii-climate-analysis-with-app

#Import Modules
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from scipy.stats import ttest_ind
plt.rcParams["figure.figsize"] = [14, 10]
import sqlalchemy
from sqlalchemy import text
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Reflect Tables into SQLAlchemy ORM

In [14]:
#Define SQLite Database Engine
engine = create_engine("sqlite:///hawaii_test.sqlite")

In [15]:
#Map Database from SQLite File
Base = automap_base()

#Import Tables from Database
Base.prepare(engine, reflect=True)

In [16]:
#Display Class Names
Base.classes.keys()

['measurement', 'station']

In [17]:
#Define Classes as Table Names
Measurement = Base.classes.measurement
Station = Base.classes.station

In [18]:
#Create Session Link to SQLite Database
session = Session(engine)

# Exploratory Climate Analysis

In [19]:
#Extract All Avialable Dates from Measurements Table
date_list = []
with engine.connect() as connection:
    result = connection.execute(text('SELECT date FROM Measurement'))
    date_list = [row for row in result.fetchall()]

#Loop Thruough Dates to Find Latest Record
for date in date_list:
    last_date = date[0]
    
#Split Lastest Date Record at Dashes to Isolate Year/Month/Day
date_split = last_date.split('-')

#Subtract One from Year Index
date_split[0] = str(int(date_split[0]) - 1)

#Re-Assemble Date Value & Record as Year Prior Date
prior_date = date_split[0] + '-' + date_split[1] + '-' + date_split[2]

#Query Database to Create Pandas Data Frame of Precipitation Data for Selected Year-Long Period
precip_data = pd.read_sql('SELECT date, prcp FROM Measurement WHERE date >= (?)', engine, params = (prior_date,))

#Drop Rows with Missing Values
precip_data = precip_data.dropna()

#Set Date Values as Data Frame Index
precip_data = precip_data.set_index('date')

#Sort Data Frame by Date
precip_data = precip_data.sort_index()

#Plot Bar Chart of Precipitation History for Selected Year
precip_data.plot(legend = None)
plt.xlabel('Date')
plt.ylabel('Precipitation Amount (in)')
plt.title('Hawaii Precipitation from ' + prior_date + ' to ' + last_date)
plt.savefig('Precipitation_History.png')
plt.show()